In [ ]:
from sklearn_som.som import SOM
from sklearn import datasets
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

In [ ]:
iris = datasets.load_iris()
iris_data = iris.data[:, :2]
iris_label = iris.target

In [ ]:
print(iris_data.shape)
print(iris_label.shape)

In [ ]:
plt.scatter(iris_data[:,0],iris_data[:,1],c=iris_label)

In [ ]:
iris_som = SOM(m=3, n=1, dim=2)
iris_som.fit(iris_data)

In [ ]:
predictions = iris_som.predict(iris_data)

In [ ]:
predictions.shape

In [ ]:
plt.scatter(iris_data[:,0],iris_data[:,1],c=predictions)

# Andiamo a utilizzare questo algoritmo di som sul nostro dataset

In [ ]:
nome_file='Friday-02-03-2018_TrafficForML_CICFlowMeter'

with open('pickle/'+nome_file+'/scaled_document.pickle', 'rb') as handle:

    df = pickle.load(handle)

In [ ]:
feature_1 = 'Init Bwd Win Byts'
feature_2 = 'Fwd Pkts/s'


df = df[[feature_1, feature_2,'Label']]

In [ ]:
df_prova = df.groupby(['Label'])['Label'].count()


df_prova=df_prova.to_frame()
print(df_prova)
df_prova.set_index('Label')
df_prova=df_prova.rename(columns={'Label':'Count'})

In [ ]:
bot = df[df.Label == 1]
benign = df[df.Label == 0]

print("Benign: ", len(benign), "Bot: ", len(bot))

In [ ]:
#bilanciamento del dataset
balanced_d = pd.concat([bot, benign.sample(len(bot))])
x = balanced_d.iloc[:,:-1]
y = balanced_d.iloc[:,-1:]
balanced_d.shape

In [ ]:
print(x.shape)
print(type(x.values))

In [ ]:
# Eseguiamo il clustering

n_clusters = len(balanced_d['Label'].unique())  # numero di cluster

y_pred = SOM(m=1, n=2, dim=2)

y_pred.fit(x.values)

predictions = y_pred.predict(x.values)

# Associazione del cluster i-esimo con la classe i-esima
cluster_class_mapping = {}
for i in range(n_clusters):
    cluster_samples = x[predictions == i]
    cluster_classes = y[predictions == i]
    unique_classes, class_counts = np.unique(cluster_classes, return_counts=True)
    dominant_class = unique_classes[np.argmax(class_counts)]
    cluster_class_mapping[i] = dominant_class

In [ ]:
difference = pd.DataFrame(data=[],columns=['num_cluster','num_class'])
print(y['Label'].unique())
print(type(y_pred))
print(np.unique(predictions))

In [ ]:
# Plot dei risultati
plt.scatter(x[feature_1], x[feature_2], c=y['Label'])
plt.title("Optimal Number of Clusters")
plt.show()

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.scatter(x[feature_1], x[feature_2], c=predictions)
axs.set_title("Mixture of Gaussian Blobs")

# Stampiamo l'associazione del cluster con la classe

for i in range(n_clusters):
    if(i != cluster_class_mapping[i]):

        new_row = pd.Series({'num_cluster': i, 'num_class': cluster_class_mapping[i]})
        difference = pd.concat([difference,new_row.to_frame().T],ignore_index=True)
        
    axs.text(
        np.mean(x[feature_1][predictions == i]), np.mean(x[feature_2][predictions == i]),
        f"Cluster {i}: Class {cluster_class_mapping[i]}",
        fontsize=12, fontweight='bold', color='red', ha='center', va='center'
    )

plt.suptitle("Ground truth clusters").set_y(0.95)
plt.show()

In [ ]:
indici = []

for a in difference['num_cluster']:
    indici.append([
    index for index in range(len(predictions))
    if predictions[index] == a
])

In [ ]:
for ind,true_value in zip(indici,difference['num_class']):
    for a in ind:
        difference[a] = true_value

In [ ]:
# Plot dei risultati
plt.scatter(x[feature_1], x[feature_2], c=y['Label'])
plt.title("Optimal Number of Clusters")
plt.show()

fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.scatter(x[feature_1], x[feature_2], c=predictions)
axs.set_title("Mixture of Gaussian Blobs")

# Stampiamo l'associazione del cluster con la classe

for i in range(n_clusters):
    if(i != cluster_class_mapping[i]):

        new_row = pd.Series({'num_cluster': i, 'num_class': cluster_class_mapping[i]})
        difference = pd.concat([difference,new_row.to_frame().T],ignore_index=True)
        
    axs.text(
        np.mean(x[feature_1][predictions == i]), np.mean(x[feature_2][predictions == i]),
        f"Cluster {i}: Class {cluster_class_mapping[i]}",
        fontsize=12, fontweight='bold', color='red', ha='center', va='center'
    )

plt.suptitle("Ground truth clusters").set_y(0.95)
plt.show()

In [ ]:
print(accuracy_score(predictions,y['Label']))